This is a PU Results Scrapper. It's a light weight code that helps you to get the result of given
roll number by giving url , code without using physical browser.
This code has certain classes that does work together to achieve the goal.

Codder: Kanwar Adnan
Email: kanwaradnanrajput@gmail.com

In [1]:
# Code by Kanwar Adnan
import bs4
import requests
import pprint
import time

In [2]:
# URLGenerator class to generate URLs for fetching student data
class URLGenerator:
    def __init__(self , url , code , roll_nums):
        # Initialize URLGenerator with URL, code, and list of roll numbers  
        self.url = url
        self.code = code
        self.roll_nums = roll_nums

    def make_urls(self):
        """
        Makes URLs out of the roll numbers for fetching data.
        """
        urls = []
        for roll_num in self.roll_nums:
            temp = {
                "link": self.url,
                "code": self.code,
                "roll_num": roll_num
            }
            urls.append(temp)
        return urls

In [3]:
# HtmlFetcher class to fetch HTML from URLs
class HtmlFetcher:
    def __init__(self , urls):
        """
        Initialize HtmlFetcher with list of URLs.
        """
        self.urls = urls
        self.null_roll_nums = []

    def fetch_html(self , url : dict):
        """
        This function fetches the html from the given url.
        We have to make a custom payload for this purpose.
        """
        payload = {
            "result_code":  url['code'],
            "roll_no": url['roll_num'],
            "submitcontact": "Submit"
            }
        return requests.post(url['link'],  data = payload).text

    def fetch_all(self, urls : list):
        """
        It's a wrapper of fetch_html. It gets all the htmls of all the urls.
        """
        htmls = []
        for url in urls:
            html = (self.fetch_html(url))
            htmls.append(html)
        return htmls

In [4]:
# DataProcessor class to process and parse data from HTML
class DataProcessor:
    def __init__(self, htmls):
        """
        Initialize DataProcessor with list of HTML strings.
        """
        self.htmls = htmls

    def generateResult(self,html):
        soup = bs4.BeautifulSoup(html , 'html.parser')
        tables = soup.findAll('table','style2')

        text = tables[0].text.split('\n')

        temp = {
            "degree" : text[3],
            'roll_no' : text[7],
            'name' : text[11],
            'father_name' : text[15],
            'reg_no' : text[19],
        }
        temp
        text = tables[1].text.split('\n')

        temp.update({
            "institute" : text[12],
            "credit" : text[13],
            "result" : text[14]
        })

        return temp


    def parse(self,html):
        """
        Generates a result
        """
        return self.generateResult(html)

    def parse_all(self):
        """
        Generates all the results
        """
        return [self.parse(html) for html in self.htmls]

In [5]:
# ResultsFetcher class to manage the process of fetching and processing results
class ResultsFetcher:
    def __init__(self, roll_nums: list, url: str = None , code: str = None):
        """ Make sure every object already has it's attributes. This is just a executor
        class. """
        self.url = url
        self.code = code
        self.roll_nums = roll_nums

    def get_urls(self):
        url_generator = URLGenerator(url = self.url, code = self.code, roll_nums = self.roll_nums)
        urls = url_generator.make_urls()

        return urls

    def fetch_htmls(self , urls: list = None):
        if not urls:
            urls = self.get_urls()
        html_fetcher = HtmlFetcher(urls)
        htmls = (html_fetcher.fetch_all(urls))

        while None in htmls:
            htmls.remove(None)

        return htmls

    def process_data(self , htmls):
        data_processor = DataProcessor(htmls)
        data = data_processor.parse_all()
        return data

    def main(self):
        # Calculating the time of completion
        start = time.time()
        htmls =(self.fetch_htmls())
        end = time.time()

        self.fetching_time = round(end - start,2)

        start = time.time()
        results = (self.process_data(htmls))
        end = time.time()

        self.parsing_time = round(end - start,2)

        return results

In [6]:
def main():
    """
    Roll Numbers Results on PU:
    44939 1st probation
    44926 last probation
    45626 late result
    44760 dropped 
    45216 promoted
    45218 has compart in previous semester
    """
    # Roll Numbers Results on PU
    roll_nums = [44939,44926,45626,44760,45216,45218]
    code = "str_bsfydp3semsf21_230922"
    url = "http://pu.edu.pk/home/results_show/7471"

    fetcher = ResultsFetcher(url = url , code = code , roll_nums = roll_nums )
    results = (fetcher.main())

    print("Fetched Resutls\n")
    pprint.pprint(results , indent = 4)
    print("")
    print("#Stats")
    print(f"Fetching Time: {fetcher.fetching_time} ")
    print(f"Parsing Time: {fetcher.parsing_time}")


In [7]:
main()

Fetched Resutls

[   {   'credit': '18',
        'degree': 'Bachelor`s Studies Chemistry',
        'father_name': 'Muhammad Akram',
        'institute': 'Government College of Science, Wahdat Road, Lahore',
        'name': 'Muhammad Abrar Ul Haq',
        'reg_no': '2020-gsr-594',
        'result': 'CGPA  1.99 1st Probation (Fail in CHEM-201, MATH-211, '
                  'ZOOL-201, re-appear till Exam 2023)',
        'roll_no': '44939'},
    {   'credit': '18',
        'degree': 'Bachelor`s Studies Chemistry',
        'father_name': 'Azhar Ghauri',
        'institute': 'Government College of Science, Wahdat Road, Lahore',
        'name': 'Zafeer Hussain',
        'reg_no': '2020-gsr-602',
        'result': 'CGPA  1.81 Last Probation (Fail in BOT-201, CHEM-201, '
                  'ZOOL-201, re-appear till Exam 2023) (Prv ...',
        'roll_no': '44926'},
    {   'credit': '18',
        'degree': 'Bachelor`s Studies Sociology',
        'father_name': 'Sagheer Ahmad',
        'institut

Isn't it amazing you just fetched the results so quickly without going to physical web-browser.
If you are thinking this can't be better than this you might be wrong.

1) What if you want to scrape a thousands of results.
   Fetching 6 results took 1 second and parsing took 0.8 seconds. so fetching 100 results will cost you 3 minutes.

2) What if the user entered a roll number that is not valid for the specific url. Then it will return None or even
   it can crash the program.

3) What if you want to do analysis on the roll numbers like how many have been promoted and dropped etc etc.

   Now it seems to be a problem in this script right.

News:
    But I have also made a super charged high speed version of this code which can handle any problem that occur and can work instantly. Getting 100 results won't cost you even a second. If you want that code just hit me a DM.